# Расчетно-графическое домашнее задание

### Выполнил: Александров А. Н., ИУ8-104
### Вариант: 1

## Задание
Задана платежная матрица прямоугольной игры с нулевой суммой.


| 4  | -3 | 5  | 6  | 4 |
|:--:|:--:|:--:|:--:|:-:|
| 6  | 5  | -3 | 4  | 7 |
| 6  | 5  | -3 | -3 | 5 |
| -3 | -3 | 4  | 4  | 4 |
| 7  | 6  | 4  | 5  | 6 |


1. Нормализовать матрицу (привести к матрице с неотрицательными элементами) и свести исходную игру к матричной игре 2×2 следующими способами:
   - поглощением доминируемых стратегий;
   - удалением NBR-стратегий (Never Best Response).
2. Найти смешанные стратегии игроков следующими методами:
   - графоаналитическим;
   - аналитическим (матричным);
   - графически (задача ЛП);
   - симплекс-методом (задача ЛП).
3. Рассчитать цену игры для исходной матрицы.


In [1]:
import json
import logging
from pathlib import Path

import numpy as np

from game_theory.utils.simplex.simplex_problem import SimplexProblem
from game_theory.utils.simplex.dual_problem import DualProblem
from game_theory.utils.matrix_games.game_matrix import GameMatrix

logging.basicConfig(level=logging.INFO, format='%(message)s')

In [2]:
# Входная матрица прямоугольной игры с нулевой суммой.
matrix = np.array(
    [
        [4, -3, 5, 6, 4],
        [6, 5, -3, 4, 7],
        [6, 5, -3, -3, 5],
        [-3, -3, 4, 4, 4],
        [7, 6, 4, 5, 6],
    ],
    dtype=int,
)

game_matrix = GameMatrix(matrix)
game_matrix

+---------------------------------------------------------+
|               Таблица стратегий (игрока А)              |
+----------------+----+----+----+----+----+---------------+
|   Стратегии    | b1 | b2 | b3 | b4 | b5 | MIN выигрыш A |
+----------------+----+----+----+----+----+---------------+
|       a1       | 4  | -3 | 5  | 6  | 4  |       -3      |
|       a2       | 6  | 5  | -3 | 4  | 7  |       -3      |
|       a3       | 6  | 5  | -3 | -3 | 5  |       -3      |
|       a4       | -3 | -3 | 4  | 4  | 4  |       -3      |
|       a5       | 7  | 6  | 4  | 5  | 6  |       4       |
| MAX проигрыш B | 7  | 6  | 5  | 6  | 7  |               |
+----------------+----+----+----+----+----+---------------+

### 1. Нормализация матрицы. Уменьшение размерности исходной матричной игры

In [3]:
game_matrix.normalize_matrix()
game_matrix

+---------------------------------------------------------+
|               Таблица стратегий (игрока А)              |
+----------------+----+----+----+----+----+---------------+
|   Стратегии    | b1 | b2 | b3 | b4 | b5 | MIN выигрыш A |
+----------------+----+----+----+----+----+---------------+
|       a1       | 7  | 0  | 8  | 9  | 7  |       0       |
|       a2       | 9  | 8  | 0  | 7  | 10 |       0       |
|       a3       | 9  | 8  | 0  | 0  | 8  |       0       |
|       a4       | 0  | 0  | 7  | 7  | 7  |       0       |
|       a5       | 10 | 9  | 7  | 8  | 9  |       7       |
| MAX проигрыш B | 10 | 9  | 8  | 9  | 10 |               |
+----------------+----+----+----+----+----+---------------+

#### 1.1. Поглощение доминируемых стратегий

**Доминирующая (поглощающая) строка** содержит элементы $\geq$ элементам другой строки (поглощаемой).

**Доминируюший (поглощающая) столбец** содержит элементы $\leq$ элементам другого столбца (поглощаемого).

In [4]:
reduced_matrix: GameMatrix = game_matrix.reduce_dimension(mode='dominant_absorption')
reduced_matrix

Поглощение стратегии a3 доминирующей стратегией a2
Поглощение стратегии a4 доминирующей стратегией a1
Поглощение стратегии b1 доминирующей стратегией b2
Поглощение стратегии b4 доминирующей стратегией b3
Поглощение стратегии b5 доминирующей стратегией b2
Поглощение стратегии a2 доминирующей стратегией a5


+------------------------------------------+
|       Таблица стратегий (игрока А)       |
+----------------+----+----+---------------+
|   Стратегии    | b2 | b3 | MIN выигрыш A |
+----------------+----+----+---------------+
|       a1       | 0  | 8  |       0       |
|       a5       | 9  | 7  |       7       |
| MAX проигрыш B | 9  | 8  |               |
+----------------+----+----+---------------+

#### 1.2. Удаление NBR-стратегий

In [5]:
reduced_matrix: GameMatrix = game_matrix.reduce_dimension(mode='nbr_drop')
reduced_matrix

NotImplementedError: Этот метод я пока не прошёл :(

### 2. Нахождение смешанных стратегий

$S_A = p_1 + p_2 + ... + p_n$ - смешанная стратегия игрока A.
$S_B = q_1 + q_2 + ... + q_m$ - смешанная стратегия игрока B.

#### 2.1. Графоаналитический метод

#### 2.2. Аналитический (матричный) метод

#### 2.3. Графический метод (задача ЛП)

#### 2.4. Симплекс-метод (задача ЛП)

<div style="text-align:center;">
    <img src="img/system_matrix_game.png" alt="system_matrix_game" width="450" height="330">
    <img src="img/LP_task_01.png" alt="LP_task_01" width="450" height="330">
</div>
<div style="text-align:center;">
    <img src="img/LP_task_02.png" alt="LP_task_02" width="450" height="330">
</div>


In [10]:
print(reduced_matrix.game_matrix.tolist())
type(reduced_matrix.game_matrix)
reduced_matrix

[[0, 8], [9, 7]]


+------------------------------------------+
|       Таблица стратегий (игрока А)       |
+----------------+----+----+---------------+
|   Стратегии    | b2 | b3 | MIN выигрыш A |
+----------------+----+----+---------------+
|       a1       | 0  | 8  |       0       |
|       a5       | 9  | 7  |       7       |
| MAX проигрыш B | 9  | 8  |               |
+----------------+----+----+---------------+

In [13]:
n_rows, n_cols = reduced_matrix.game_matrix.shape
input_data = {
    "obj_func_coffs": [1] * n_cols,
    "constraint_system_lhs": (reduced_matrix.game_matrix * (-1)).tolist(),
    "constraint_system_rhs": [-1] * n_rows,
    "func_direction": "min"
}

input_path = Path('input.json')
input_path.write_text(json.dumps(input_data))

132

##### 2.4.1. Двойственная задача ЛП для игрока A

In [14]:
dual_problem = DualProblem(input_path)

F = cx -> max,
Ax >= 1,
x1, x2, ..., xn >= 0
C = [1 1]
A =
[[ 0 -9]
 [-8 -7]],
b^T = [1 1].


In [15]:
dual_problem.solve()

Процесс решения:
Поиск опорного решения:
Исходная симплекс-таблица:
+----+------+------+------+
|    | Si0  |  x1  |  x2  |
+----+------+------+------+
| x3 | -1.0 | 0.0  | 9.0  |
| x4 | -1.0 | 8.0  | 7.0  |
| F  | 0.0  | -1.0 | -1.0 |
+----+------+------+------+


SimplexProblemException: Задача не имеет допустимых решений! При нахождении опорного решения не нашлось отрицательного элемента в строке с отрицательным свободным членом.

##### 2.4.2. Прямая задача ЛП для игрока B

In [16]:
simplex_problem = SimplexProblem(input_path)
simplex_problem

Условие задачи:
Найти вектор x = (x1,x2,..., xn)^T как решение след. задачи:
F = cx -> min,
Ax <= b,
x1,x2, ..., xn >= 0
C = [1 1],
A =
[[ 0 -8]
 [-9 -7]],
b^T = [-1 -1].

In [17]:
simplex_problem.solve()

Процесс решения:
Поиск опорного решения:
Исходная симплекс-таблица:
+----+------+------+------+
|    | Si0  |  x1  |  x2  |
+----+------+------+------+
| x3 | -1.0 | 0.0  | -8.0 |
| x4 | -1.0 | -9.0 | -7.0 |
| F  | 0.0  | -1.0 | -1.0 |
+----+------+------+------+
Разрешающая строка: x3
Разрешающий столбец: x2
+----+--------+------+--------+
|    |  Si0   |  x1  |   x3   |
+----+--------+------+--------+
| x2 | 0.125  | -0.0 | -0.125 |
| x4 | -0.125 | -9.0 | -0.875 |
| F  | 0.125  | -1.0 | -0.125 |
+----+--------+------+--------+
Разрешающая строка: x4
Разрешающий столбец: x1
+----+--------+---------+---------+
|    |  Si0   |    x4   |    x3   |
+----+--------+---------+---------+
| x2 | 0.125  |   -0.0  |  -0.125 |
| x1 | 0.0139 | -0.1111 |  0.0972 |
| F  | 0.1389 | -0.1111 | -0.0278 |
+----+--------+---------+---------+
Опорное решение найдено!
x4 = x3 = 0, 
x2 = 0.12, x1 = 0.01
Целевая функция: F = 0.14
Поиск оптимального решения:
Оптимальное решение найдено!
x4 = x3 = 0, 
x2 = 0.12

0.1389